<a href="https://colab.research.google.com/github/caturpramono/tugas6_auto/blob/main/tugas6_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================================
# INTERAKTIF DEMO: Exponential Moving Average, Kalman Filter,
# dan Savitzky-Golay Filter
# ==========================================================

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from ipywidgets import interact, FloatSlider, IntSlider

# ----------------------------
# 1. Buat sinyal asli + noise
# ----------------------------
fs = 1000
t = np.linspace(0, 1, fs)
signal_asli = np.sin(2 * np.pi * 5 * t)             # sinyal 5 Hz
noise = np.random.normal(0, 0.4, len(t))            # noise acak
signal_noisy = signal_asli + noise

# ----------------------------
# 2. Definisi filter
# ----------------------------

# a. Exponential Moving Average
def exponential_moving_average(data, alpha):
    ema = np.zeros_like(data)
    ema[0] = data[0]
    for i in range(1, len(data)):
        ema[i] = alpha * data[i] + (1 - alpha) * ema[i-1]
    return ema

# b. Kalman Filter sederhana (tanpa library tambahan)
def kalman_filter(data, process_var=1e-3, measurement_var=0.1):
    x_est = 0.0    # estimasi awal
    P = 1.0        # error covariance awal
    R = measurement_var
    Q = process_var
    filtered = []
    for z in data:
        # Prediksi
        P = P + Q
        # Update
        K = P / (P + R)
        x_est = x_est + K * (z - x_est)
        P = (1 - K) * P
        filtered.append(x_est)
    return np.array(filtered)

# c. Savitzky-Golay Filter
def savgol_filter_custom(data, window_length, polyorder):
    if window_length % 2 == 0:
        window_length += 1  # harus ganjil
    return savgol_filter(data, window_length, polyorder)

# ----------------------------
# 3. Fungsi plotting interaktif
# ----------------------------
def demo_filter(alpha=0.2, process_var=1e-3, measurement_var=0.1, window_sg=15, poly_sg=3):
    # Terapkan filter
    signal_ema = exponential_moving_average(signal_noisy, alpha)
    signal_kalman = kalman_filter(signal_noisy, process_var, measurement_var)
    signal_savgol = savgol_filter_custom(signal_noisy, window_sg, poly_sg)

    plt.figure(figsize=(12, 8))

    plt.subplot(4,1,1)
    plt.plot(t, signal_noisy, color='gray')
    plt.title("Sinyal Asli + Noise")
    plt.ylabel("Amplitudo")
    plt.grid(True)

    plt.subplot(4,1,2)
    plt.plot(t, signal_noisy, color='lightgray')
    plt.plot(t, signal_ema, color='orange', label=f"EMA (α={alpha})")
    plt.legend(); plt.grid(True); plt.title("Exponential Moving Average")

    plt.subplot(4,1,3)
    plt.plot(t, signal_noisy, color='lightgray')
    plt.plot(t, signal_kalman, color='purple', label=f"Kalman Filter (Q={process_var}, R={measurement_var})")
    plt.legend(); plt.grid(True); plt.title("Kalman Filter")

    plt.subplot(4,1,4)
    plt.plot(t, signal_noisy, color='lightgray', label="Input Noisy")
    plt.plot(t, signal_savgol, color='green', label=f"Savitzky-Golay (win={window_sg}, poly={poly_sg})")
    plt.legend(); plt.grid(True); plt.title("Savitzky-Golay Filter")

    plt.tight_layout()
    plt.show()

# ----------------------------
# 4. Slider kontrol interaktif
# ----------------------------
interact(
    demo_filter,
    alpha=FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='EMA α'),
    process_var=FloatSlider(value=1e-3, min=1e-5, max=1e-1, step=1e-4, description='KF Process Var'),
    measurement_var=FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='KF Meas Var'),
    window_sg=IntSlider(value=15, min=3, max=101, step=2, description='SG Window'),
    poly_sg=IntSlider(value=3, min=1, max=7, step=1, description='SG Poly'),
)


interactive(children=(FloatSlider(value=0.2, description='EMA α', max=1.0, min=0.01, step=0.01), FloatSlider(v…

<function __main__.demo_filter(alpha=0.2, process_var=0.001, measurement_var=0.1, window_sg=15, poly_sg=3)>